# API Generate Picture:  

In [2]:
#export REPLICATE_API_TOKEN = ************************

Connect API with Replicate's model - instruct-pix2pix. This model can help generate the image based on the user input instruction and parameters. <br>
the image is the link: https://image.uniqlo.com/UQ/ST3/us/imagesgoods/465185/item/usgoods_71_465185.jpg?width=700<br>
Some parameters explaination:
- Prompt: prompt to guide the image generation. 
- negative_prompt: The prompt or prompts not to guide the image generation (what you do not want to see in the generation). 
- num_outputs: Number of images to output
- guidance_scale: Scale for classifier-free guidance. Higher guidance scale encourages to generate images that are closely linked to the text prompt, usually at the expense of lower image quality. (I higher this a little bit, default is 7.5)
- num_inference_steps: The number of denoising steps. More denoising steps usually lead to a higher quality image at the expense of slower inference.
- image_guidance_scale: Image guidance scale is to push the generated image towards the inital image. Higher image guidance scale encourages to generate images that are closely linked to the source image, usually at the expense of lower image quality.

In [1]:
import replicate
from IPython.display import Image

output = replicate.run(
    "timothybrooks/instruct-pix2pix:30c1d0b916a6f8efce20493f5d61ee27491ab2a60437c13c588468b9810ec23f",
    input={
        "image": "https://image.uniqlo.com/UQ/ST3/us/imagesgoods/465185/item/usgoods_71_465185.jpg?width=700",
        "prompt": "Change only the T-shirt's color to pink.",
        "negative_prompt": """Don't change the pant's color!!!! Don't change the background color!!!!""",
        "scheduler": "K_EULER_ANCESTRAL",
        "num_outputs": 1,
        "guidance_scale": 9,
        "num_inference_steps": 100,
        "image_guidance_scale": 1.5
    }
)

# Extract the URL of the output image
output_image_url = output[0]  # Assuming the output is a list containing the URL of the image

# Display the image in the notebook
Image(url=output_image_url, width=300, height=200)


# Prompt Engineering

## Langchain

This is the LLMChain model from langchain. An LLMChain is a simple chain that adds some functionality around language models. It is used widely throughout LangChain, including in other chains and agents.It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.<br>

Variables in the code:
- customized_text: this contains a template for generating marketing promotion content. This template includes user input metrices such as product name, features, hashtags, weather, advertising objectives, promotion specials, and style of writing. It also includes variables such as image, query, and an answer. 
- template: the prompt template has the pdf file we created (template_text) and the customized_text.

Model explaination:<br>
- Prompt Template: The LLMChain typically operates with a prompt template. This template serves as a structured **input format that guides the model** in understanding and generating text. It contains placeholders for variables that can be filled with specific information relevant to the task at hand.
- LLMChain: The LLMChain includes one or more language model instances. These instances are responsible for **processing the prompt template and generating text based on the provided input**. In the provided code, the LLMChain utilizes an instance of ***OpenAI's GPT-4 model*** for text generation.

Overall, the LLMChain model acts as an intermediary layer between the user input and the language model(s), providing a structured approach to leverage the power of language models for specific tasks while allowing flexibility in input formats and model selection. It enables the composition of more sophisticated language processing pipelines by chaining together multiple language models and processing steps.

In [3]:
# !pip install openai==0.27.8 langchain==0.0.225

In [7]:
import openai
import os

os.environ['OPENAI_API_KEY'] = '******************************'

In [5]:
#pip install PyPDF2

In [8]:
# Read the pdf file:
import PyPDF2

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

pdf_file_path = 'instruction.pdf'

template_text = extract_text_from_pdf(pdf_file_path)

template = template_text

In [9]:
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain

davinci = OpenAI(model_name='gpt-4')

customized_text = """
Generate marketing promotion content.
You will receive a picture named "{image}" please identify the following user input matrices:

Product Name: Uniqlo AIRism Cotton Crew Neck T-Shirt
Features: Breathable, quick-drying, anti-odor
Hashtags: #UniqloOutfit #UniqloFittingRoom".
Weather: Warm, spring
Advertising Objective: Promotion
Promotion Specials:  2 for $29.90 
Style of Writing: For KOLs

Question: {query}

Answer: 
"""

# Build prompt template for simple question-answering
template = template_text + customized_text

prompt = PromptTemplate(input_variables=["query", "image"], template=template)

llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

question = "Please write marketing content based on the template"
image_file_name = "image.jpg"  # Provide the image file name here

print(llm_chain.run(query=question, image=image_file_name))


Title: "Get in gear for warm weather with Uniqlo's AIRism Cotton Crew Neck T-Shirt!🌤️"

Text: "Spring is here, and it's time to dress accordingly. Presenting Uniqlo's AIRism Cotton Crew Neck T-Shirt - your perfect companion for the changing weather🌸. Crafted with breathable, quick-drying, and anti-odor properties, this t-shirt is designed for the active individual on the go. 🏃‍♂️

Whether you're meeting friends for a weekend brunch or running errands in the city, this shirt has got you covered with its seamless fit and advanced AIRism technology. Best part? It's part of our current promotion, where you can buy 2 for $29.90! 🙌

Why wait? Visit your nearest Uniqlo store or shop online now to avail of this limited-time offer! Don't forget to tag us in your fits using #UniqloOutfit and #UniqloFittingRoom. Excited to see how you rock this super cool and comfy tee! 👕✨

#UniqloAIRism #UniqloPromotion @UNIQLO.fit.fanpage @UniqloOutfitGuide"


## Evaluate Performance

In [21]:
#pip install -U sentence-transformers

### Real case test

In [18]:
#Real case test:
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain

davinci = OpenAI(model_name='gpt-4')

customized_text = """
Generate marketing promotion content.
You will receive a picture named "{image}" please identify the following user input matrices:

Product Name: Uniqlo sun protection cardigan
Features: outdoor, comfortable, refreshing, Loose fit, Slimming, freely
Hashtags: #UniqloFittingRoom #UniqloBestsellersDiary #UniqloOutfit #Uniqlo #UniqloFashionLightPants #UniqloWorkPants
Weather: summer
Advertising Objective: 
Promotion Specials:  
Style of Writing: For KOLs

Question: {query}

Answer: 
"""

# Build prompt template for simple question-answering
template = template_text + customized_text

prompt = PromptTemplate(input_variables=["query", "image"], template=template)

llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

question = "Please write marketing content based on the template"
image_file_name = "test_set/images/5.jpg"  # Provide the image file name here

print(llm_chain.run(query=question, image=image_file_name))


/Users/carol/anaconda3/lib/python3.11/site-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/carol/anaconda3/lib/python3.11/site-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Title: This is THE perfect cardigan you need for summer! ☀️ 

Text: 
Just in time for the summer heat, the Uniqlo sun protection cardigan has got you covered. This beauty is more than just a coverup. It's lightweight, breathable, and oh-so-chic, perfect for out-and-about adventures! 🌴
  
The loose yet slimming fit gives you that comfortable, airy feel without sacrificing style. Fantastic, right? Pair it with your favorite Uniqlo work pants, and you're all set. It's your ultimate summer staple in the wardrobe. 👖

With its refreshing cut, not only does it protect you from the sun's glare, it also elevates your look. Style effortlessly and stays cool in this summer must-have piece! 🌸 Effortless and stylish, because that’s the Uniqlo way.

Get yours in Uniqlo now, don't miss out on this summer must-have. 🛍️  See ya in #UniqloFittingRoom! 📸 #Uniqlo #UniqloBestsellersDiary #UniqloOutfit #UniqloFashionLightPants #UniqloWorkPants


In [20]:
# Evaluate Performance:
test_dataset = [
    {"ground_truth": "[Title]: Uniqlo | Worthy of me getting two colors of the sun protection cardigan❗️ Extremely simple and relaxed.\n[Text]: Just eagerly looking forward to spring/summer outfits. I really love these two pairs of pants, they're the lightest work pants I've ever worn. Loose fit, covering and slimming, making you look taller. Girls with a bit of belly fat really love pants with drawstring designs at the waist, they're very comfortable. The cuffs can also be adjusted freely, allowing for both wide-leg and tapered styles. The light gray one even has sun protection, feels comfortable and refreshing to wear. The knee area is thoughtfully pleated, so there's no feeling of restriction no matter how you move. Suitable for outdoor activities or daily wear. Really good, love them ❤️ Tried them on, very 👍 Sisters, go for it with confidence! 👖: 464888, 468583 #UniqloFittingRoom #UniqloBestsellersDiary #UniqloOutfit #Uniqlo #UniqloFashionLightPants #UniqloWorkPants #UniqloParachutePants #UniqloPants #SpringSummerOutfit #WorkPantsOutfit #VersatileWorkPants #MiracleSlimmingPants #DailyOutfit #EverydayOutfit"}
    # Add more test cases as needed
]#real case test

from sentence_transformers import SentenceTransformer, util
sentences = [test_dataset, llm_chain.run(query=question, image=image_file_name)]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)

tensor([[0.6097]])

## Few Shot

Variables in the code:
- customized_text: this contains a template for generating marketing promotion content. This template includes user input metrices such as product name, features, hashtags, weather, advertising objectives, promotion specials, and style of writing. It also includes variables such as query and an answer.
- query: template_text and customized_text together is the query (template)

Few-Shot Prompt Template:<br>
The FewShotPromptTemplate class is used for few-shot learning, where the model learns from a few examples provided. It takes examples of prompts containing queries, answers, and picture filenames. These examples serve as training data for the model.<br>
It utilizes the defined PromptTemplate to format the examples and includes a prefix and suffix for contextual information around the examples.<br>
The example_separator parameter specifies how examples are separated in the template.<br>

Prefix:<br>
- The prefix is a piece of text added to the beginning of the input text. It can be used to introduce context, provide additional information, or set the tone for the generated text.
- In the context of text generation, a prefix can help guide the model by providing context or constraints for the generated text. It can specify the topic, style, or desired outcome of the text generation process.
- For example, in a conversational setting, a prefix might include information about the speaker or the context of the conversation.

Suffix:<br>
- The suffix is a piece of text added to the end of the input text. It can be used to **conclude the input text**, provide closing remarks, or prepare the model for the next part of the text.
- Similar to the prefix, the suffix can influence the model's output by providing context or indicating the expected structure of the generated text.
- For example, in a dialogue generation task, a suffix might include a prompt for the next speaker or signal the end of the conversation.
--> I did not put any prefix here, but put the user input metrices into the suffix.<br>



In [8]:
from langchain import FewShotPromptTemplate, PromptTemplate


# Create example prompts
examples = [
    {
        "query": template_text,
        "answer": "",
        "picture_filename": "image.jpg"  # Placeholder filename for the picture
    },
]

example_template = """
User: {query}
AI: {answer}
Picture filename: {picture_filename}
"""


example_prompt = PromptTemplate(
    input_variables=["query", "answer", "picture_filename"],  # Make sure all input parameters are included
    template=example_template
)

#  It can be used to introduce the response or provide any contextual information. 
prefix = """"""

#  It can be used to conclude the response or add any closing remarks.
suffix = """please identify the following user input matrices:

Product Name: Uniqlo AIRism Cotton Crew Neck T-Shirt
Features: Breathable, quick-drying, anti-odor
Hashtags: #UniqloOutfit #UniqloFittingRoom".
Weather: Warm, spring
Advertising Objective: Promotion
Promotion Specials:  2 for $29.90 
Style of Writing: For KOLs

User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\\\\n\\\\n"
)

# Define a sample query
query = "Can you provide marketing content for this UNIQLO product based on the given picture?"

print(
    davinci(
        few_shot_prompt_template.format(query=query)
    )
)



Sure, based on your provided user input matrix and the image analyzed, let me generate the marketing content for the Uniqlo AIRism Cotton Crew Neck T-Shirt. 

Content Structure: Target Audience - Unisex, Product - Uniqlo AIRism Cotton Crew Neck T-Shirt, Season - Spring 

🟢 **Title (For KOLs)**: "Feel the air on your skin with Uniqlo AIRism Cotton Crew Neck T-Shirt! 🌤🍃"

**Body Text (For KOLs)**: 

"Hello sunshine and hello spring!🌤🌷 For all of you out there ready to embrace the warm season, there's no better way to start your spring wardrobe than with the Uniqlo AIRism Cotton Crew Neck T-Shirt.💚 🙌

Made of breathable, quick-drying, and anti-odor material, you will love the effortless comfort that comes with it. The T-Shirt keeps you fresh as a daisy all day long, even as the temperature soars! 😎💦

Hit the streets or relax at home, either way, you'll love the softness of our AIRism on your skin! It's so lightweight that it almost feels like you're not wearing anything🍃

This amazing T-S

## Sequential Chains prompting

The SequentialChain allows us to combine multiple chains sequentially, creating an integrated chain. This is useful when we want to apply a series of transformations or operations to the input data.<br>

To illustrate the use of generic chains, let's go through an example workflow in which we will:<br>
1. We have a dirty input text with extra spaces.
2. We pass the input text through the clean_extra_spaces_chain to remove the extra spaces.
3. We then pass the cleaned text to the style_paraphrase_chain to paraphrase the text in a specific style (e.g., a poet or a policeman).
<br>

 
Functions Explaination:<br>
- replace_newlines: Replaces newline characters in the text.
- count_tokens: Counts the number of tokens used during model execution.
- transform_func: Performs text transformations to clean extra spaces and newlines.
- clean_extra_spaces_chain: A transformation chain to clean extra spaces.
- customized_text: Customized text for marketing promotion content generation.
- template: Prompt template for generating paraphrases.
- style_paraphrase_chain: An LLMChain for paraphrasing text.
- sequential_chain: A SequentialChain to apply transformations and paraphrasing sequentially.
- input_data: Dictionary containing input text, style, and image filename.
- count_tokens: Executes the SequentialChain with the input data and counts the number of tokens used.

In [11]:
import re  
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains import TransformChain, SequentialChain  

def replace_newlines(text):
    # Replace '\n' with actual newline characters
    text = text.replace(r'\n', '\n')
    return text

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
        print(replace_newlines(result))
#     return result 

def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    # Replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\\\\r\\\\n|\\\\r|\\\\n){2,}', r'\\\\n', text)
    text = re.sub(r'[ \\\\t]+', ' ', text)
    return {"output_text": text}

def add_image_to_input(inputs: dict) -> dict:
    # Assuming the image is provided as a filename in the 'image' key
    image_filename = inputs["image"]
    # Here you can load the image and process it as needed
    # For simplicity, let's just pass the filename to the output
    return {"image": image_filename}

# Initialize OpenAI's GPT model
davinci = OpenAI(model_name='gpt-4')

# Define a transformation chain to clean extra spaces
clean_extra_spaces_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output_text"],
    transform=transform_func
)

customized_text = """
Generate marketing promotion content.
You will receive a picture named "{image}" 

{output_text}
 
In the style of a {style}.

Paraphrase:

"""

# Define a PromptTemplate for generating paraphrases
template = template_text + customized_text
prompt = PromptTemplate(input_variables=["style", "output_text", "image"], template=template)

# Define an LLMChain for paraphrasing
style_paraphrase_chain = LLMChain(
    llm=davinci,
    prompt=prompt,
    output_key='final_output'
)

# Define a SequentialChain to apply transformations and paraphrasing sequentially
sequential_chain = SequentialChain(
    chains=[clean_extra_spaces_chain, style_paraphrase_chain],
    input_variables=['text', 'style', 'image'],
    output_variables=['final_output']
)

input_text = """
Please generate marketing content based on the template requirements.

please identify the following user input matrices:

Product Name: Uniqlo AIRism Cotton Crew Neck T-Shirt
Features: Breathable, quick-drying, anti-odor
Hashtags: #UniqloOutfit #UniqloFittingRoom".
Weather: Warm, spring
Advertising Objective: Promotion
Promotion Specials:  2 for $29.90 
Style of Writing: For KOLs
"""

# Specify the input text and image filename
input_data = {'text': input_text, 'style': 'Ready for Some Spring Fun with Uniqlo\'s AIRism T-Shirts! #UniqloOutfit', 'image': 'image.jpg'}

# Run the SequentialChain with the input data
count_tokens(sequential_chain, input_data)


Spent a total of 2697 tokens
Title: Spring is in the Air with Uniqlo's AIRism! 🌸#UniqloFittingRoom 

Text: Hello there, fashion lovers! 😘 Let's welcome the warm weather with Uniqlo's AIRism Cotton Crew Neck T-Shirt. 🌞👕 Designed to keep you cool and comfortable, this shirt is your perfect companion for spring. Its breathable, quick-drying, and anti-odor features are heaven-sent for sunny days out! 🏝️ For those of you who like to mix and match, good news! We’ve got a special promotion - 2 for just $29.90! This is a golden opportunity you surely don't want to miss. Grab yours now and let's jazz up our spring wardrobe. 💃🕺Follow us to the #UniqloFittingRoom and let's get you fitted right away. Share your spring #UniqloOutfit, we can't wait to see your stylish picks!  🥳


## Differences among models

### LLMChain:<br>
- The LLMChain, or Language Model Chain, is a construct used to chain together multiple language models (LLMs) or other processing steps to perform complex NLP tasks.
- It typically consists of one or more language models (e.g., GPT, BERT) and possibly other processing steps like tokenization, encoding, or decoding.
- The purpose of an LLMChain is to process input text sequentially through a series of language models or processing steps to achieve a specific NLP task, such as text generation, summarization, or translation.

In my code:
- the LLMChain is utilized to generate marketing promotion content based on a given prompt template and input variables.
- It uses OpenAI's GPT-4 model (davinci) to process the prompt template and generate text output.
- The LLMChain is responsible for processing the prompt template, filling in the input variables, and generating text output based on the model's predictions.

### FewShotPromptTemplate:<br>
- The FewShotPromptTemplate is a template-based approach used for few-shot learning in NLP tasks.
- It involves providing a small number of example prompts along with their corresponding inputs and outputs to train a model to perform a particular task.
- The purpose of a FewShotPromptTemplate is to enable models to generalize from a few examples and adapt to new tasks or prompts with limited training data.
- It's particularly useful when training data is scarce or expensive to collect, as it allows models to learn from a small number of examples.

In my code:<br>
- The FewShotPromptTemplate concept is relevant because the LLMChain is trained and operates with a template 
- These examples serve as training data for the model, allowing it to adapt to different styles or prompts when generating text.

### SequentialChain:
- The SequentialChain is a concept used to sequentially apply a series of transformations or processing steps to input data.
- It can involve a combination of data preprocessing, feature engineering, and model inference steps.
- The purpose of a SequentialChain is to orchestrate the sequential execution of processing steps to prepare input data for model inference or to perform multi-step tasks that involve data transformations.

In my code: 
- a SequentialChain is created to apply text transformations (cleaning extra spaces) and then paraphrase the text based on a specific style.
- The SequentialChain ensures that the input text is preprocessed before being passed to the language model for further processing.